In [1]:
import pickle
from sklearn.manifold import MDS, TSNE
import numpy
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as pyplot
import gensim
from sklearn.cluster import KMeans
import time

#load word2vec
model = gensim.models.KeyedVectors.load_word2vec_format('cc.fr.300.vec.bin', binary=True)

#load categories centers
with open('final_tags/final_tags.pickle', 'rb') as f:
        average_vectors,closest_words,final_tags,y_kmeans = pickle.load(f)
cat_kmean = KMeans(n_clusters = 400)    
cat_kmean.cluster_centers_ = average_vectors        


In [2]:
manual_tags = ['','','','','','','Local','','','','','','Course','','','','','','','Nature','Religion','','Actualité',
'','','Actualité','','','','Sport','','','','','','','Parcs','','','Local','Championnats','','','','',
'Local','','','Politique municipale','','Justice','','Politique','','','','','','','Lieu','','',
'Routes & Circulation','','','','','','','','','Internationnal','','Éducation','','Famille','','','','Art','','','','',
'Politique','Local','','','Routes & Circulation','','Santé','','Routes & Circulation','','','Blessures & Hospitalisation','','','',
'Détente','','','Local','Sport','','','','','','','','','','','Local','','Musique','','Local',
'','','','','Sports d''hiver','Éducation','Chant','','','','Société','Local','Hockey','','','','','','Sport','',
'Noël','','Sport','Projets','','','Restauration','','Politique','','Local','','','','','','','','',
'Cinéma','','Animaux','','Polluants & Dangers','','Crimes & Accidents','','Internationnal','Spectacles','','','','','','','','','Gastronomie','',
'Taxes','Célébration','','Musique','','Culture','','','Infrastructures','','','','','Animaux','','','Local','',
'Nature','Statistiques & Tendances','Nature','','','','','Maladie & Mortalité','','','','','','','','Crimes & Accidents','Politique','','Éducation','Santé',
'Gastronomie','','','Agriculture','','','','Drogues & Alcool','','Justice','','','Météo','','','','','Sport',
'Internationnal','','Finance','','','Industrie','','Télévision','','','','','Local','Hiver','','Véhicules','','','',
'','','','','Crimes & Accidents','','','','Gastronomie','Nature','','','','','','','Musique','','Cinéma','','Météo','','','',
'Nature','Art','','Économie','Système de santé','','','','Local','Jeux','Musique','Politique','','Télévision',
'Routes & Circulation','','','','','','','','','','Politique','','Spectacles','','','','','','','','','','','','',
'','Hebdomadaire','','','Finance','Crimes & Accidents','Politique','Véhicules','Justice','','','','','','','','','','Justice','',
'','Famille','','','','','Véhicules','Spectacle','','','','Local','Gastronomie','Politique','','','','Polluants & Dangers','',
'Hebdomadaire','Sport','Finance','','','','','','Mode','','','','USA','Incendies','','Grèves','','','Radio','','',
'Électricité','Hebdomadaire','','','','','','','Économie','','','','Hockey','','','Cyclisme','Nature','','',
'','','','','']

In [3]:
#load les vecteurs et textes des articles à classer
good_words = []
word_freq = []
word_vectors = []
bad_words = []
for i in range(50000):
    with open('vectors/article_'+str(i+0)+'.pickle', 'rb') as f:
        this_word,this_freq,this_vec,this_bad_word=pickle.load(f)
        good_words.append(this_word)
        word_freq.append(this_freq)
        word_vectors.append(this_vec)
        bad_words.append(this_bad_word)
        
data_clean = []
initial_tags = []

for i in range(50000):
    with open('printable/article_'+str(i+0)+'.pickle', 'rb') as f:
        this_data_clean, this_initial_tags = pickle.load(f)
        data_clean.append(this_data_clean)
        initial_tags.append(this_initial_tags)

In [5]:
a = [0.48]
b = [4]
mean_categories = numpy.zeros((len(a),len(b)))
no_categories = numpy.zeros((len(a),len(b)))

for a_i in range(len(a)):
    for b_i in range(len(b)):
        #Détermine les principaux centres de l'article
        all_main_centers = []
        all_assigned_clusters = []
        all_cluster_centers = []
        all_cluster_weigths = []
        all_main_i = []

        for i in range(len(word_vectors)):
            if i%100==99:
                print(i)

            X = numpy.array(word_vectors[i])
            weights = numpy.array(word_freq[i])
            words = numpy.array(good_words[i])
            
            if len(X)<=4:
                all_main_centers.append(X)
                all_assigned_clusters.append(list(range(len(X))))
                all_cluster_centers.append(X)
                all_cluster_weigths.append(weights)
                all_main_i.append(list(range(len(X))))
                continue
            
            N = round(len(X)**(a[a_i]))

            #kmeans
            k=round(len(X)/b[b_i])
            if k<1:
                k=1
            kmean = KMeans(n_clusters = k)    
            kmean.fit(X, sample_weight = weights)
            y = kmean.predict(X)

            cluster_weights = []
            cluster_centers = kmean.cluster_centers_
            for j in range(k):
                cluster_weights.append(sum(weights[y==j]))
            main_i = sorted(range(len(cluster_weights)), key = lambda sub: cluster_weights[sub])[-N:] 
            main_centers = cluster_centers[main_i]

            all_main_centers.append(main_centers)
            all_assigned_clusters.append(y)
            all_cluster_centers.append(cluster_centers)
            all_cluster_weigths.append(cluster_weights)
            all_main_i.append(main_i)

        main_centers = all_main_centers
        assigned_clusters = all_assigned_clusters
        cluster_centers = all_cluster_centers
        cluster_weights = all_cluster_weigths
        main_i = all_main_i





        #assigner les catégories aux articles
        X_list = []
        for article in main_centers:
            for center in article:
                X_list.append(center)
        X = numpy.zeros((len(X_list),300))
        for i in range(len(X_list)):
            X[i] = X_list[i]
        y_kmeans = cat_kmean.predict(X)
        i = 0
        final_tags = []
        for article in main_centers:
            final_tags.append([])
            for center in article:
                final_tags[-1].append(y_kmeans[i])
                i=i+1



        #Assigner les tags correspondants aux catégories
        all_manual_tags = []
        len_all_manual_tags = []
        n=0
        for i in range(len(final_tags)):
            all_manual_tags.append([])
            for tag in final_tags[i]:
                if manual_tags[tag]!='':
                    if manual_tags[tag] in all_manual_tags[i]:
                        pass
                    else:
                        all_manual_tags[i].append(manual_tags[tag])
            len_all_manual_tags.append(len(all_manual_tags[i]))
            if len(all_manual_tags[i])==0:
                all_manual_tags[i].append('Variété')
                n=n+1
        no_categories[a_i, b_i] = n
        mean_categories[a_i, b_i] = numpy.mean(numpy.asarray(len_all_manual_tags))
        


99
199
299
399
499
599
699
799
899
999
1099
1199
1299
1399
1499
1599
1699
1799
1899
1999
2099
2199
2299
2399
2499
2599
2699
2799
2899
2999
3099
3199
3299
3399
3499
3599
3699
3799
3899
3999
4099
4199
4299
4399
4499
4599
4699
4799
4899
4999
5099
5199
5299
5399
5499
5599
5699
5799
5899
5999
6099
6199
6299
6399
6499
6599
6699
6799
6899
6999
7099
7199
7299
7399
7499
7599
7699
7799
7899
7999
8099
8199
8299
8399
8499
8599
8699
8799
8899
8999
9099
9199
9299
9399
9499
9599
9699
9799
9899
9999
10099
10199
10299
10399
10499
10599
10699
10799
10899
10999
11099
11199
11299
11399
11499
11599
11699
11799
11899
11999
12099
12199
12299
12399
12499
12599
12699
12799
12899
12999
13099
13199
13299
13399
13499
13599
13699
13799
13899
13999
14099
14199
14299
14399
14499
14599
14699
14799
14899
14999
15099
15199
15299
15399
15499
15599
15699
15799
15899
15999
16099
16199
16299
16399
16499
16599
16699
16799
16899
16999
17099
17199
17299
17399
17499
17599
17699
17799
17899
17999
18099
18199
18299
18399
18499
1

In [75]:

print(mean_categories)
print(no_categories)

[[2.10322]]
[[3959.]]


In [11]:
#Checkpoint

#for i in range(50000):
#    with open('clusters_test/article_'+str(i)+'.pickle', 'wb') as f:
#        pickle.dump([all_main_centers[i],all_assigned_clusters[i],all_cluster_centers[i],all_cluster_weigths[i],all_main_i[i]], f)

main_centers = []
assigned_clusters = []
cluster_centers = []
cluster_weights = []
main_i = []

for i in range(50000):#598927+1
    with open('clusters_test/article_'+str(i+0)+'.pickle', 'rb') as f:
        this_main_centers, this_assigned_clusters, this_cluster_centers, this_cluster_weights, this_main_i=pickle.load(f)
        main_centers.append(this_main_centers)
        assigned_clusters.append(this_assigned_clusters)
        cluster_centers.append(this_cluster_centers)
        cluster_weights.append(this_cluster_weights)
        main_i.append(this_main_i)

#assigner les catégories aux articles
X_list = []
for article in main_centers:
    for center in article:
        X_list.append(center)
X = numpy.zeros((len(X_list),300))
for i in range(len(X_list)):
    X[i] = X_list[i]
y_kmeans = cat_kmean.predict(X)
i = 0
final_tags = []
for article in main_centers:
    final_tags.append([])
    for center in article:
        final_tags[-1].append(y_kmeans[i])
        i=i+1
        
 #Assigner les tags correspondants aux catégories
all_manual_tags = []
len_all_manual_tags = []
n=0
for i in range(len(final_tags)):
    all_manual_tags.append([])
    for tag in final_tags[i]:
        if manual_tags[tag]!='':
            if manual_tags[tag] in all_manual_tags[i]:
                pass
            else:
                all_manual_tags[i].append(manual_tags[tag])
    len_all_manual_tags.append(len(all_manual_tags[i]))
    if len(all_manual_tags[i])==0:
        all_manual_tags[i].append('Variété')
        n=n+1


In [13]:
#Afficher le résultat
for i in range(25000,25010): #article
    time.sleep(0.004)
    print('--------------ARTICLE #'+str(i)+'--------------')
    print(data_clean[i])
    print('--------------TAG INITIAL--------------')
    print(initial_tags[i])
    print('--------------TAGS FINAUX--------------')
    print(all_manual_tags[i])
    #print('--------------TAGS ML--------------')
    #for m,j in enumerate(final_tags[i]):
    #    print(m)
    #    print(closest_words[j])
    #    print('\n')
    #print('--------------WORDS IN CLUSTER--------------')
    #for m,j in enumerate(main_i[i]):
    #    print(m)
    #    print('\n')
    #    temp_string = ''
    #    for k,is_in_cluster in enumerate(assigned_clusters[i]==j):
    #        if is_in_cluster:
    #            temp_string = temp_string+good_words[i][k]+', '
    #    print(temp_string)
    print('\n\n________________________________________________________________________________________\n')

--------------ARTICLE #25000--------------
 Cinq heures de belles trouvailles

 L'engouement pour l'activité est tel, qu'elle est maintenant organisée deux fois par année ce qui permet d'amasser un montant substantiel, bien que modeste considérant les besoins.

 Tout au long de la journée, plusieurs dizaines de personnes se sont réunies dans la grand Hall de La Pulperie de Chicoutimi où les 182 lots étaient présentés, soit 16 de plus que l'an dernier. L'encan a d'ailleurs été devancé d'une heure, en raison du grand nombre de lots proposés.

 «On parle de cinq heures d'encan. C'est un véritable marathon pour l'encanteur. Notre objectif est de vendre pour environ 30 000$. On termine avec un bénéfice du tiers», explique Sonia Lemay, directrice générale des Entreprises-Jeunesse.

 Cette année, le public a notamment pu faire l'acquisition d'oeuvres d'Arthur Villeneuve, Jean-Paul Lapointe, Louis Tremblay, Albert Rousseau, René Gagnon, Christian Bergeron et Marcellin Dufour.

 Plusieurs antiq

In [26]:
initial_names = ['Sports','Sport','Remparts','LNH','Économie','Économie','Affaires','Éducation','Actualités',
                 'Actualité','Politique','Politique municipale',
                 'Politique fédérale','Cyclisme',
                'Restaurants','Santé']
assigned_names = ['Sport','Sport','Sport','Sport','Économie','Finance','Économie','Éducation','Actualité',
                  'Actualité','Politique','Politique municipale',
                  'Politique','Cyclisme',
                 'Gastronomie','Santé']
for i in range(len(initial_names)):
    n_tag = 0
    n_well_assigned = 0
    initial_name=initial_names[i]
    assigned_name = assigned_names[i]
    for i in range(25000,50000):
        if initial_name in initial_tags[i]:
            n_tag=n_tag+1
            if assigned_name in all_manual_tags[i]:
                n_well_assigned=n_well_assigned+1
    print('Occurence de ',assigned_name,' dans ',initial_name,': ',n_well_assigned/n_tag*100,'%')



Occurence de  Sport  dans  Sports :  46.91286785920369 %
Occurence de  Sport  dans  Sport :  46.43843180563225 %
Occurence de  Sport  dans  Remparts :  82.52427184466019 %
Occurence de  Sport  dans  LNH :  72.22222222222221 %
Occurence de  Économie  dans  Économie :  28.499999999999996 %
Occurence de  Finance  dans  Économie :  10.75 %
Occurence de  Économie  dans  Affaires :  34.22131147540984 %
Occurence de  Éducation  dans  Éducation :  33.17972350230415 %
Occurence de  Actualité  dans  Actualités :  7.484792382967469 %
Occurence de  Actualité  dans  Actualité :  7.858637526156707 %
Occurence de  Politique  dans  Politique :  68.23161189358372 %
Occurence de  Politique municipale  dans  Politique municipale :  25.0 %
Occurence de  Politique  dans  Politique fédérale :  68.75 %
Occurence de  Cyclisme  dans  Cyclisme :  12.121212121212121 %
Occurence de  Gastronomie  dans  Restaurants :  68.42105263157895 %
Occurence de  Santé  dans  Santé :  31.186440677966104 %


In [27]:
all_tags = []
for tag in initial_tags:
    if tag not in all_tags:
        all_tags.append(tag)
print(len(all_tags))

all_tags = []
for tag in manual_tags:
    if tag not in all_tags:
        all_tags.append(tag)
print(len(all_tags))

1246
61


In [15]:
n_occurences_tags = []
for i in range(len(closest_words)):
    print('---------------------')
    print(manual_tags[i])
    print(i)
    print(sum(y_kmeans==i))
    n_occurences_tags.append(sum(y_kmeans==i))
    print(closest_words[i])

---------------------

0
6112
peuvent, vont, doivent, viennent, retrouvent, devraient, trouvent, sont, demandent, voient, font, passent, veulent, parviennent, continuent, pourront, semblent, prennent, reconnaissent, arrivent, 
---------------------

1
18562
donc, surtout, alors, même, autant, aussi, mais, tout, fait, pourtant, cependant, effectivement, évidemment, certainement, toujours, encore, juste, néanmoins, justement, semble-t-il, 
---------------------

2
5862
trois, quatre, deux, cinq, six, huit, douze, dix, treize, quinze, sept, onze, vingt, plusieurs, quatres, quatorze, dix-huit, seize, vingt-deux, vingt-cinq, 
---------------------

3
11044
plus, moins, assez, peu, trop, encore, suffisamment, très, nettement, relativement, beaucoup, tellement, extrêmement, étonnamment, particulièrement, toujours, plsu, guère, autant, aussi, 
---------------------

4
165
sq, sq., ft, mq, sqm, sqq, b-c, sq.m, sqq., mtrs, Kč, ft2, Sq, sqft, sq.m., sq.ft., sv, m2, m., nm., 
---------------------

936
ottawa, uottawa, dOttawa, ottawa.ca, ontario, toronto, mcgill, rimouski, Ottawa, sherbrooke, québec, uottawa.ca, saguenay, québec., longueuil, uqam, chicoutimi, lOntario, Ottawa., gatineau, 
---------------------

46
597
selon, dont, Selon, puisque, que, même, laquelle, indépendamment, lequel, avec, mais, Ainsi, car, lesquelles, Sachant, parce, sachant, alors, lorsque, comme, 
---------------------

47
10340
michel, philippe, claude, nicolas, laurent, eric, franck, christophe, alain, andrée, mathieu, patrick, girard, stéphane, bernard, bernier, gérald, phillipe, marcelle, jean-françois, 
---------------------
Politique municipale
48
1326
maire, Maire, maires, député-maire, maire-adjoint, élus, municipal, édile, sénateur-maire, ex-maire, mairie, député, députée-maire, sous-préfet, maire., Sénateur-maire, élu, Sénateur-Maire, Député-Maire, Député-maire, 
---------------------

49
984
jean, jean., jean-, jeans, marcel, monnet, jardinbleu1107, blue-jean, claude, jacques, yanne, pantalo

416
côté, coté, autre, derrière, passage, aspect, juste, coté-ci, dessus, arrière, côtés, côté-ci, côté-là, coin, opposé, face, extrémité, -côté, part, côté.Un, 
---------------------
Routes & Circulation
88
2586
véhicule, camion, voiture, véhicules, camion-remorque, autobus, voitures, chauffeur, taxi, camions, bateau, remorqueuse, camionnette, avion, engin, volant, bus, minibus, routier, automobile, 
---------------------

89
28
of, Of, the, and, in, ofthe, to, for, my, de, is, In, by, At, with, DE, zu, For, du, To, 
---------------------
Santé
90
769
santé, santés, santé5, santéet, santé., santée, Santé, santé1, sante, santé.La, santé3, santé4, santé.De, santé.Une, nutrition, lasanté, santé.Si, santé.Pour, santépublique, santéÀ, 
---------------------

91
178
an, An, an., an.Un, 2an, Bao', Hui', an.Et, 365j, 1an, an.Ce, Rui', 365J, anLe, an.-, ecto1, Dao', an-, Qor', an.Il, 
---------------------
Routes & Circulation
92
583
route, chemin, routes, route.De, Route, route.La, autoroute,

5754
québec, saguenay, sherbrooke, montréal, québec., chicoutimi, rimouski, longueuil, montreal, drummondville, quebec, shawinigan, granby, gaspésie, terrebonne, laurentides, toronto, canada, charlevoix, victoriaville, 
---------------------
Hockey
131
188
lnh, nhl, lhjmq, nfl, sags, mlb, Nhl, nba, chsb, mvp, lch, mhsc, u16, u18, lfp, dtn, tfc, 2K9, lnr, u17, 
---------------------

132
597
monde, entier, tout, monde.Tout, monde.Ce, monde.Un, monde., monde.Il, monde.Et, monde.Le, monde.Mais, Monde, continent, mondeet, pays, monde.Au, mond, monde.On, monde.Même, monde-là, 
---------------------

133
11
ha, ha., Ha, 40ha, Mha, 000ha, 20ha, hℓ, 18ha, hect, 50ha, 11ha, 16ha, 50Ha, 10ha, 40Ha, 9ha, 15ha, 13ha, 5ha, 
---------------------

134
434
blanc, noir, gris, bleu, brun, rouge, marron, jaune, beige, blancle, doré, vert, violet, blanc.On, blanc.Un, blanc., bleu.Le, blanc.Le, argenté, rose, 
---------------------

135
88
mario, Mario, wario, luigi, mariokart, marios, -Mario, nintendo, r

214
cas, cas.Dans, cas.En, sauf, cas., cas.Au, cas.Ce, cas.Si, cas.Un, cas-là, cas.Mais, cas.D', cas.Par, revanche, cas.Donc, cas.Et, cas.Après, Sauf, cas.Alors, cas.De, 
---------------------

172
1558
trop, plus, assez, peu, très, suffisamment, moins, tellement, relativement, excessivement, beaucoup, extrêmement, encore, trèèès, suffisament, vraiment, extrèmement, nettement, trèèèèèèèès, bien, 
---------------------

173
12
cm, cm., cmx, cm5, 39cm, cm.A, cm.Il, 1,7cm, 13cm, mm, 98cm, 44cm, 37cm, cm24, cmA, 83cm, cm.1, 73cm, 56cm, 11cm, 
---------------------

174
39
f, λf, yf, 1-f, .g, λx, .f, F, d,, ⁡, v,, r-r, r, 1-v, b, Δf, 2.8D, cking, .u, hN, 
---------------------

175
36
van, Van, den, Tuyl, der, Eerd, G.Van, Baaren, Harten, Houdt, Gilse, Aerde, R.Van, vanden, Grieken, von, Tuyll, Zanden, zegen, M.Van, 
---------------------
Gastronomie
176
72
café, thé, café., café.Le, cafe, café.-, café-, caféL', café.La, Café, expresso, café.L', capuccino, thé., café1, caffé, cappuccino, Th

383
cours, test, tests, début, semestre, programme, cours.Un, examens, pré-stage, inter-cours, niveau, cours., exercices, cursus, mois, cycle, durant, mini-cours, Cours, stage, 
---------------------
Santé
215
172
soins, soin, soins., desoins, palliatifs, Soins, soins-, traitements, infirmiers, médicaux, paliatifs, médico-esthétiques, soins.Les, soins.Pour, santé, prodigués, soinsL, soignants, soins.Le, services, 
---------------------
Gastronomie
216
118
vin, vins, Vin, vins., crémants, crémant, vignoble, vins.Le, rosé, champagne, vin., Vins, liquoreux, vin.Le, pouilly-fumé, vignobles, spiritueux, pommard, crus, vins.La, 
---------------------

217
8608
une, cette, première, nouvelle, grande, importante, traditionnelle, toute, seule, celle-ci, partie, histoire, contemporaineémission, petite, dernière, perspective, façon, structure, telle, belle, 
---------------------

218
1013
six, cinq, huit, quatre, douze, dix, trois, sept, quinze, treize, onze, dix-huit, vingt, quatorze, vingt-deu

4220
jeune, musicien, homme, écrivain, père, comédien, artiste, romancier, chanteur, cinéaste, garçon, journaliste, poète, soldat, politicien, acteur, personnage, ami, peintre, réalisateur, 
---------------------

255
10
fr, nl, FR, wp, 〈, /, fr-, uk, www.vacancesbleues.fr, hln, obp, eu1, www.diplomatie.gouv.fr, smf, fr-FR, Thnks, rcl, fr-CA, esBESbswy, en-gb, 
---------------------

256
76
amp, &, co, u0026, lt, gt, Co, eacute, rsquo, nbsp, x20AC, ;, x201C, xA0, x2026, apos, 039, xFB, co., eacu, 
---------------------
Crimes & Accidents
257
300
blessés, blessé, morts, blessées, tués, bléssés, blessés., blessée, grièvement, tuées, blessé., blesses, blessés.La, tué, décédés, victimes, blessés.Le, contusionnés, blessures, bléssé, 
---------------------

258
968
saison, série, saisons, mini-saison, saison.La, estivale, année, mi-saison, saiso, saison., demie-saison, semaine, hivernale, période, saison.Le, dernière, demi-saison, saison.C', Saison, saison-ci, 
---------------------

259
33


4830
ville, maison, quartier, centre-ville, village, immeuble, bâtiment, centre, jouxtant, habitation, bourg, résidence, rue, église, bâtisse, résidentiel, région, ruelle, résidentielle, lotissement, 
---------------------

297
35
p, .p, P, <, rno, LCLC, q-, pp, qh, q, PHRF, ⊨, rq, p-, 4q, v, n,, p., kq, o, 
---------------------

298
2419
peut, doit, pourra, devrait, pourrait, devra, saurait, puisse, doive, devait, semble, veut, sera, être, pouvait, est, pourait, devrai, voudra, voudrait, 
---------------------

299
1676
bernier, dupuy, duhamel, girard, giraud, pujol, martineau, dubourg, blais, guillou, roussel, benard, lemay, simard, laroche, reynaud, bocquet, dufresne, boyer, delpech, 
---------------------

300
46
size, Size, size., oahu, sized, SIZE, 8pt, sizes, king, 14px, FONT-SIZE, color, ltr, 360px, maxsize, x-small, quote, 12pt, queen, KIMBO-, 
---------------------
Politique
301
2847
gouvernement, socialiste, libéral, électorale, parlement, électoral, élection, socialistes, 

2
rd, rg, Gér, e-c, Rôge, Rhéby, Laüs, rn, uje, Buès, Còrrec, Héuré, rx, gws, Dôa, Lène, Aïse, Léès, kd, Léez, 
---------------------

340
61
perd, gagne, perdra, bat, perdent, perde, regagne, perds, reperd, abandonne, récupère, prend, retire, cède, reconquiert, reprend, conquiert, retrouve, perdait, perdu, 
---------------------

341
207
bon, excellent, mauvais, beau, bel, Bon, bon.Un, joli, vrai, bon.Bon, bien.Bon, bien, bien.Un, petit, meilleur, mieux.Un, super, mieux.Bon, tout, bonne, 
---------------------
Véhicules
342
81
taxi, bus, taxis, taxi., chauffeur, taxiLe, taxi-bus, autobus, minibus, moto-taxi, mototaxi, mini-bus, chauffeurs, Taxi, microbus, moto-taxis, vélo-taxi, taximan, tuk-tuks, tuk-tuk, 
---------------------
Spectacle
343
134
gala, Gala, bal, galas, dîner, banquet, soirée, diner, soirée-bénéfice, concert, spectacle, cérémonie, concert-bénéfice, spectacle-bénéfice, show, souper-bénéfice, anti-gala, souper, cocktail, événement-bénéfice, 
---------------------

344
50

273
qh, yf, trv, zg, ql, qj, cq, qy, xb, aV, y4, 0c, wj, fj, beq, rng, 0i, fq, bck, bsn, 
---------------------

381
132
nuit, nuit.La, nuit., nuit.Mais, nuit.De, nuit.Il, nuitA, nuite, nuit.Une, nuit.A, nuit.Ce, journée, nuit.L, nuit.Et, nuit.On, nuits, nuit-là, nuit.C', nuit.Le, soirée, 
---------------------

382
6656
les, autres, ces, certains, deux, des, plusieurs, nombreux, différents, trois, tous, quelques, derniers, mêmes, ses, innombrables, certaines, quatre, grands, petits, 
---------------------

383
101
avis, Avis, avis.A, avis., avisL', commentaires, avis.Je, avis.Il, avis.Un, apréciation, avis.Et, avisje, avis-, avis.J', avis.Si, avisÀ, avis.Le, avis.L', commentaire, propos, 
---------------------
Économie
384
1985
salaire, montant, payer, payé, repaie, impôt, salaires, versement, argent, revenus, rembourser, paye, loyer, remboursement, impôts, côtisations, paie, dépense, paiement, revenu, 
---------------------

385
49
vue, point, imprenable, vue., vue.La, vue.C', perspe

In [21]:
for i in range(40000,40001):
    print('--------------ARTICLE #',i,'--------------')
    print(data_clean[i])
    print('--------------TAG INITIAL--------------')
    print(initial_tags[i])
    print('--------------CLUSTERS DE L''ARTICLE--------------')
    for j in main_i[i]:
        temp_string = ''
        for k,is_in_cluster in enumerate(numpy.asarray(assigned_clusters[i])==j):
            if is_in_cluster:
                temp_string = temp_string+good_words[i][k]+', '
        print('-',temp_string)
    print('--------------CATÉGORIES ASSIGNÉES--------------')
    for j in final_tags[i]:
        print('#',j,'-',manual_tags[j],': ',closest_words[j])
    print('--------------TAGS FINAUX--------------')
    print(all_manual_tags[i])

--------------ARTICLE # 40000 --------------
 Les Mustangs couronnés

 Les nouveaux champions ont toutefois fait face à une vive lutte de la part des Lynx du Pavillon Wilbrod-Dufour d'Alma (gagnants de 2006) qui les ont talonnés jusqu'à la toute fin, ne concédant la bannière que par six petits points d'écart. La délégation de l'École polyvalente Arvida a pris le 3e rang, tandis que celle du Séminaire de Chicoutimi clôt la marche au 4e rang. Malheureusement, l'équipe de Mashteuiatsh, qui en aurait été à une deuxième participation au défi, a dû déclarer forfait à la dernière minute.

 Cela dit, comme l'événement s'est déroulé sous la thématique du cinéma, chaque équipe de jeunes de secondaire 1 et 2 a eu droit à un accueil de ''stars'' à leur arrivée à la mezzanine du Pavillon sportif de l'UQAC, alors que des gardes du corps en veston, cravates et lunettes fumées, les attendaient à l'entrée du tapis rouge. Un animateur interviewait en direct quelques «vedettes» qui seraient en action dur